In [23]:
tokenizer.convert_tokens_to_ids(["GGGGGG"])

[4100]

In [1]:
from datasets import load_from_disk, Dataset
from transformers import AutoTokenizer
import pandas as pd

In [5]:
dataset_path = "/s/project/semi_supervised_multispecies/all_fungi_reference/fungi/Annotation/Sequences/AAA_Concatenated/pretokenized/downstream300"
dataset_path = "/s/project/semi_supervised_multispecies/all_fungi_reference/fungi/Annotation/Sequences/AAA_Concatenated/pretokenized/speciesdownstream300"
#dataset_path = "../batch_embed"
#dataset_path = "../full_embed"
#dataset_path = "../batch"
dataset = load_from_disk(dataset_path)

In [43]:
dataset = dataset["train"]

In [35]:
dataset[1].keys()

dict_keys(['species_name', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask'])

In [45]:
tokenizer = AutoTokenizer.from_pretrained("gagneurlab/SpeciesLM", revision="downstream_species_lm")
start_species_ids = tokenizer.convert_tokens_to_ids(["GGGGGG"])[0]
# add a species token column
species_col = dataset["species_name"]
species_ids = tokenizer.convert_tokens_to_ids(species_col)
species_ids = [species_id - start_species_ids for species_id in species_ids]
# delete the species token in the input_ids
for i, input_ids in enumerate(dataset["input_ids"]):
    del input_ids[1]
    dataset["input_ids"][i] = input_ids

dataset = dataset.add_column("species_ids", species_ids)

dataset.save_to_disk("../batch_test")

Saving the dataset (0/1 shards):   0%|          | 0/16 [00:00<?, ? examples/s]

In [3]:
test_path = "../fullds_speciesembed"
test_ds = load_from_disk(test_path)
test_ds

Dataset({
    features: ['species_name', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask', 'species_id'],
    num_rows: 11463375
})

In [13]:
small_frame = pd.DataFrame(test_ds[:10])
print(small_frame.iloc[0])

species_name                                      aspergillus_clavatus
__index_level_0__                                                    0
input_ids            [2, 1855, 3312, 946, 3770, 2778, 2908, 3427, 1...
token_type_ids       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
attention_mask       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...
species_id                                                           1
Name: 0, dtype: object


In [11]:
test_ds.save_to_disk("../fullds_speciesembed")

Saving the dataset (0/43 shards):   0%|          | 0/11463375 [00:00<?, ? examples/s]

In [7]:
test_ds = test_ds.rename_column("species_ids", "species_id")

In [9]:



tokenizer = AutoTokenizer.from_pretrained("gagneurlab/SpeciesLM", revision="downstream_species_lm")

# 5447 is the highest token id
start_species_ids = tokenizer.convert_tokens_to_ids(["GGGGGG"])[0]

def add_species_input(dataset):
    # add species column
    df = pd.DataFrame(dataset)
    def gen_species_list(r):
        print(r)
        species = r["species_name"]
        species_id = tokenizer.convert_tokens_to_ids([species])[0]
        species_id -= start_species_ids
        num_rows = len(r["input_ids"])
        species_id = [species_id] * num_rows
        return species_id


    df["species_id"] = df.apply(lambda r : gen_species_list(r), axis=1)
    return Dataset.from_pandas(df)

sliced = add_species_input(sliced)
#dataset["train"] = add_species_input(dataset["train"])
#dataset["test"] = add_species_input(dataset["test"])

sliced.save_to_disk("../fullds_speciesembed")

species_name                                      aspergillus_clavatus
__index_level_0__                                                    0
input_ids            [2, 4101, 1855, 3312, 946, 3770, 2778, 2908, 3...
token_type_ids       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
attention_mask       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...
Name: 0, dtype: object
species_name                                      aspergillus_clavatus
__index_level_0__                                                    1
input_ids            [2, 4101, 1814, 3145, 279, 1102, 299, 1182, 61...
token_type_ids       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
attention_mask       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...
Name: 1, dtype: object
species_name                                      aspergillus_clavatus
__index_level_0__                                                    2
input_ids            [2, 4101, 1248, 881, 3512, 1746, 2876, 3298, 8...
token_type_ids       [0, 0, 0, 

Saving the dataset (0/1 shards):   0%|          | 0/10 [00:00<?, ? examples/s]

In [65]:
type(dataset)

datasets.arrow_dataset.Dataset